# 1.1 First NGSolve example

Let us solve the Poisson problem.

$$\text{find: } u \in H_{0,D}^1 \quad \int_\Omega \nabla u \nabla v = \int_\Omega f v, \quad \text{ for all } v \in H_{0,D}^1.$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

#### 2. Generate an unstructured mesh

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh.nv, mesh.ne   # number of vertices & elements 

(39, 56)

* Here we prescribed a maximal mesh-size of 0.2 using the `maxh` flag. 

* The mesh can be viewed by switching to the `Mesh` tab in the Netgen GUI. 

#### 3. Declare a finite element space:

In [3]:
fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space

133

Python's help system displays further documentation.

In [4]:
help(fes)

Help on H1 in module ngsolve.comp object:

class H1(FESpace)
 |   Keyword arguments can be:
 |  order: int = 1
 |    order of finite element space
 |  complex: bool = False
 |  dirichlet: regexpr
 |    Regular expression string defining the dirichlet boundary.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet = 'top|right'
 |  definedon: Region or regexpr
 |    FESpace is only defined on specific Region, created with mesh.Materials('regexpr')
 |    or mesh.Boundaries('regexpr'). If given a regexpr, the region is assumed to be
 |    mesh.Materials('regexpr').
 |  dim: int = 1
 |    Create multi dimensional FESpace (i.e. [H1]^3)
 |  dgjumps: bool = False
 |    Enable discontinuous space for DG methods, this flag is needed for DG methods,
 |    since the dofs have a different coupling then and this changes the sparsity
 |    pattern of matrices.
 |  
 |  Method resolution order:
 |      H1
 |      FESpace
 |      pybind11_builtins.pybind11_object
 |    

#### 4. Declare test function, trial function, and grid function 

* Test and trial function are symbolic objects - called `ProxyFunctions` -  that help you construct bilinear forms (and have no space to hold solutions). 

* `GridFunctions`, on the other hand, represent functions in the finite element space and contains memory to hold coefficient vectors.

In [5]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

A shorter command for obtaining at once both trial and test variables is now available: 

In [6]:
u, v = fes.TnT()

#### 5. Define and assemble linear and bilinear forms:

In [7]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(x*v)
f.Assemble()

You can examine the linear system in more detail:

In [8]:
print(f.vec)

 0.000333333
 0.0090505
 0.00633333
 0.000770891
 0.00419003
 0.00873851
 0.0110113
 0.0118831
 0.0154585
 0.0172657
 0.0153513
 0.0180248
 0.0203006
 0.00930857
 0.00722848
 0.00380471
 0.000687974
 0.000703175
 0.00144642
 0.00174641
 0.0130156
 0.0196218
 0.0235625
 0.0209991
 0.032979
 0.0275741
 0.0284488
 0.0170589
 0.0176338
 0.0124153
 0.0051304
 0.00284308
 0.00630915
 0.0220354
 0.0157804
 0.0210096
 0.0100578
 0.0181327
 0.0217548
 -6.66667e-05
 -3.33333e-05
 -0.000536217
 -0.000585765
 -0.00111116
 -0.0008
 -0.000766667
 -8.25863e-05
 -2.35728e-05
 -0.000125108
 -0.000290565
 -0.000209771
 -0.000451004
 -0.00040564
 -0.000661732
 -0.000800436
 -0.000504522
 -0.000864384
 -0.000967242
 -0.000947681
 -0.000970631
 -0.000782608
 -0.00113705
 -0.00131313
 -0.000661412
 -0.00146165
 -0.00135486
 -0.000642205
 -0.00125381
 -0.0012366
 -0.00157121
 -0.00143302
 -0.000438352
 -0.00133842
 -0.000991598
 -0.000377208
 -0.000778967
 -0.000731847
 -0.000237359
 -0.000666891
 -0.0005362

In [9]:
print(a.mat)

Row 0:   0: 1
Row 1:   1: 0.829127
Row 2:   2: 1
Row 3:   3: 0.880753
Row 4:   0: -0.5   4: 1.90755
Row 5:   4: -0.559177   5: 1.91948
Row 6:   5: -0.37742   6: 1.7799
Row 7:   1: -0.222678   6: -0.298266   7: 1.83885
Row 8:   1: -0.223343   8: 1.90243
Row 9:   8: -0.375377   9: 1.75223
Row 10:   9: -0.283322   10: 1.78161
Row 11:   2: -0.5   10: -0.268388   11: 1.99954
Row 12:   2: -0.5   11: -0.098129   12: 1.84322
Row 13:   12: -0.211616   13: 1.75354
Row 14:   13: -0.319115   14: 1.77273
Row 15:   3: -0.359363   14: -0.356559   15: 1.85161
Row 16:   3: -0.354152   16: 1.80192
Row 17:   16: -0.146234   17: 1.85831
Row 18:   17: -0.280268   18: 1.8809
Row 19:   0: -0.5   4: -0.159147   18: -0.497592   19: 1.89108
Row 20:   4: -0.689223   5: -0.134026   18: -0.213492   19: -0.734339   20: 3.5107
Row 21:   5: -0.848855   6: -0.338846   20: -0.607596   21: 3.57058
Row 22:   6: -0.76537   7: -0.325948   21: -0.590849   22: 3.53156
Row 23:   1: -0.383106   7: -0.991955   8: -1.08169   22:

#### 6. Solve the system:

In [10]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)

The Dirichlet boundary condition constrains some degrees of freedom. The argument `fes.FreeDofs()` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:

In [11]:
print(gfu.vec)

       0
       0
       0
 0.0923019
       0
       0
       0
       0
       0
       0
       0
       0
 0.0578968
 0.086337
 0.0954017
 0.0944824
 0.0888267
 0.078059
 0.0595638
 0.0331316
 0.0428238
 0.0391783
 0.0329405
 0.0190228
 0.0396205
 0.0437688
 0.0470417
 0.0751353
 0.0891838
 0.0930866
 0.0914133
 0.0837669
 0.0706131
 0.0597334
 0.0673558
 0.0730976
 0.0844105
 0.0647765
 0.0814921
       0
 -0.00563511
       0
       0
 0.0213178
       0
 -0.0350904
 0.00259849
 -0.00728873
 -0.00280466
       0
 -0.00967023
 -0.0123906
       0
 -0.0240987
 -0.0135412
       0
 -0.0171313
 -0.0200732
 -0.00676928
 -0.0212666
       0
 -0.0254979
 -0.00995785
       0
 -0.0390824
 -0.013713
       0
 -0.0269058
 -0.0176991
 -0.0333808
 -0.0238911
 -0.0243047
 -0.00442431
 -0.0102753
 -0.0146405
 -0.006598
 -0.00679179
 -0.00570818
 -0.00931926
 -0.00609179
 -0.00771503
 -0.00532019
 -0.00755367
 0.00262541
 -0.00112314
 -0.00818072
 0.00105745
 0.000896631
 -0.00820643
 -0.000779

## Ways to interact with NGSolve

* A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.


* You can write an entire python script in a text editor and call python on the command line. (A script of the above is provided in `poisson.py`.)
    ```
    python3 poisson.py
    ```
  
* If you want the Netgen GUI, then use `netgen` on the command line:
    ```
    netgen poisson.py
    ```
  You can then ask for a python shell from the GUI's menu options (`Solve -> Python shell`).
  